In [1]:
from networkx import DiGraph
import networkx as nx
from itertools import chain
chainit = chain.from_iterable
from aocutils.grid import TURNRIGHT, TURNLEFT, move, gridneigh
E = (0,1); W = (0,-1); N = (-1,0); S = (1,0)
winds = [N, E, S, W]

In [7]:
grid, neigh = gridneigh("in.txt", to_dict=True, parser=lambda x: [(ch) for ch in x])

def getneigh(coor, wind, rev=False):
    turn = [((coor, TURNLEFT(wind)), 1000), ((coor, TURNRIGHT(wind)), 1000)]
    newloc = move(coor, wind)
    if rev:
        newloc = move(coor, {N:S, S:N, E:W, W:E}[wind])
    if newloc in grid and grid[newloc] != '#':
        turn.append(((newloc, wind), 1))
    return turn


start = ([k for k,v in grid.items() if v == 'S'].pop(), E)
goal = [k for k,v in grid.items() if v == 'E'].pop()

g = DiGraph()
g.add_weighted_edges_from([
    ((loc,w),*n) 
        for loc in grid if grid[loc] != '#' 
        for w in winds 
        for n in getneigh(loc, w)
    ])


for wind in winds:
    print(
            nx.shortest_path_length(g, start, (goal, wind), 'weight'),
        len(
            set(step[0]
                  for l in nx.all_shortest_paths(g, start, (goal, wind), 'weight')
                  for step in l)
            )
          )

99416 471
98416 471
99416 471
100416 471


In [9]:
#| export
import heapq
def dijkstra(connections,start, goal=None):
    """
    Requires a dict with as values a LIST of tuples (neighbor, weight)
    Or a function returning a list of tuples with neighbors and weights per node

    Returns
    if goal == None:    return all paths from start
    elif goal found:    returns path to goal
    else:               returns False
    """
    seen = set() # the locations that have been explored
    frontier = [(0,start)] # the locations that still need to be visited
    parents = {start: (None,0)}

    while frontier:
        # print('\n\n',frontier,'\n',parents)
        search_cost, search_node = heapq.heappop(frontier)
        # print('looking for', search_node,search_cost)
        if search_node == goal: break
        neighbors = connections(*search_node)
        if neighbors:
            for n in neighbors:
                if n[0] not in parents or n[1]+ search_cost < parents[n[0]][1]:
                    heapq.heappush(frontier,(search_cost+n[1],n[0]))
                    parents[n[0]]= (search_node,search_cost+n[1])
        seen.add(search_node)
    if goal is None: return parents

    return parents



In [10]:
# grid
grid, neigh = gridneigh("t0.txt", to_dict=True, parser=lambda x: [(ch) for ch in x])

start = ([k for k,v in grid.items() if v == 'S'].pop(), E)
goal = [k for k,v in grid.items() if v == 'E'].pop()
goals = {(goal, w) for w in winds}

parents = dijkstra(getneigh, start, goals)
# locs = {loc for loc, wind in bestlocs}
# tocheck = {(move(loc, wind), {N:S, S:N, E:W, W:E}[wind]) for loc, wind in goals if grid[move(loc, wind)] != '.' }
# bestscore

In [11]:
def get_path(parents,goal):
    cur = goal
    path = [cur]
    cost = parents[cur][1]
    while cur != start:
        cur = parents[cur][0]
        path.append(cur)
    path.reverse()
    return path,cost
res =  {g: get_path(parents,g)[1] for g in goals}
from aocutils.common import sortdict
bestscore = min(res.values())
tocheck = [(loc, bestscore) for loc, score in sortdict(res) if score == bestscore]
tocheck

[(((1, 13), (-1, 0)), 7036)]

In [12]:
# how I initially solved it was wrong, I blocked each obstacle and ran dijkstras again
parents = dijkstra(getneigh, start, goals)
(bestlocs, bestscore) = get_path(parents, goal)
locs = {loc for loc, wind in bestlocs}
for loc in {loc for loc, wind in bestlocs}:
    grid[loc] = '#'
    a = dijkstra(getneigh, start, goals)
    if a != False:
        newlocs, newscore = a
        if newscore == bestscore:
            locs |= {loc for loc, wind in newlocs}
    grid[loc] = '.'
len(locs)
    

KeyError: (1, 13)